In [27]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


import warnings
warnings.filterwarnings("ignore")

In [28]:
drop_out_val = 0.05

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
        ) 

        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=8, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
        ) 

        self.pool1 = nn.MaxPool2d(2, 2) 

        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
        ) 

        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
        ) 

        self.pool2 = nn.MaxPool2d(2, 2) 
        self.convblock100 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(1, 1), padding=0, bias=False),
        ) 

        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
          )

        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
        ) 

        self.gap = nn.Sequential(nn.AvgPool2d(3)) 
        self.fc = nn.Linear(16, 10) 


    def forward(self, x):
        x = self.convblock1(x)
        x = F.layer_norm(x, (x.size()[1:]))
        x = F.dropout(x, p=drop_out_val)
        x = self.convblock2(x)
        x = F.layer_norm(x, (x.size()[1:]))
        x = F.dropout(x, p=drop_out_val)
        x = self.pool1(x)
        x = self.convblock3(x)
        x = F.layer_norm(x, (x.size()[1:]))
        x = F.dropout(x, p=drop_out_val)
        x = self.convblock4(x)
        x = F.layer_norm(x, (x.size()[1:]))
        x = F.dropout(x, p=drop_out_val)
        x = self.pool2(x)
        x = self.convblock100(x)
        x = self.convblock5(x)
        x = F.layer_norm(x, (x.size()[1:]))
        x = F.dropout(x, p=drop_out_val)
        x = self.convblock6(x)
        x = F.layer_norm(x, (x.size()[1:]))
        x = F.dropout(x, p=drop_out_val)
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.log_softmax(x)

In [29]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
              ReLU-2            [-1, 8, 28, 28]               0
            Conv2d-3            [-1, 8, 28, 28]             576
              ReLU-4            [-1, 8, 28, 28]               0
         MaxPool2d-5            [-1, 8, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]           1,152
              ReLU-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           2,304
              ReLU-9           [-1, 16, 14, 14]               0
        MaxPool2d-10             [-1, 16, 7, 7]               0
           Conv2d-11              [-1, 8, 7, 7]             128
           Conv2d-12             [-1, 16, 5, 5]           1,152
             ReLU-13             [-1, 16, 5, 5]               0
           Conv2d-14             [-1, 1

In [30]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [31]:
from tqdm import tqdm


def train(model, device, train_loader, optimizer, epoch, train_loss_graph, train_accuracy_graph):
    model.train()
    train_loss = 0
    correct = 0
    # pbar = tqdm(train_loader)
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
    train_loss /= len(train_loader.dataset)
    accuracy = 100. * correct / len(train_loader.dataset)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        train_loss, correct, len(train_loader.dataset), accuracy
        ))
    
    train_loss_graph.append(train_loss)
    train_accuracy_graph.append(accuracy)
    return train_loss_graph, train_accuracy_graph, accuracy

incorrect_examples = []
def test(model, device, test_loader, test_loss_graph, test_accuracy_graph):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            # print(output.shape, target.shape)
            # _, pred = torch.max(output,1)
            # idxs_mask = ((pred == target) == False).nonzero()
            # incorrect_examples.append((data[idxs_mask].cpu()).numpy())

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),accuracy
        ))
    test_loss_graph.append(test_loss)
    test_accuracy_graph.append(accuracy)
    return test_loss_graph, test_accuracy_graph


In [32]:
# from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import ReduceLROnPlateau

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=0, verbose=True)

train_loss_graph = []
train_accuracy_graph = []
test_loss_graph = []
test_accuracy_graph = []
for epoch in range(1, 16):
    print('epoch number ', epoch)
    train_loss_graph, train_accuracy_graph, accuracy = train(model, device, train_loader, optimizer, epoch, train_loss_graph, train_accuracy_graph)
    scheduler.step(accuracy)
    test_loss_graph, test_accuracy_graph = test(model, device, test_loader, test_loss_graph, test_accuracy_graph)

import matplotlib.pyplot as plt

# print(train_loss_graph, test_loss_graph)
# print(train_accuracy_graph, test_accuracy_graph)
# plt.figure(figsize=(10,5))
# plt.title("Training and Test Loss")
# plt.plot(train_loss_graph,label="train_loss")
# plt.plot(test_loss_graph,label="test_loss")
# plt.xlabel("iterations")
# plt.ylabel("Loss")
# plt.legend()
# plt.show()

# plt.figure(figsize=(10,5))
# plt.title("Training and Test Accuracy")
# plt.plot(train_accuracy_graph,label="train_accuracy")
# plt.plot(test_accuracy_graph,label="test_accuracy")
# plt.xlabel("iterations")
# plt.ylabel("Accuracy")
# plt.legend()
# plt.show()


epoch number  1
Train set: Average loss: 0.0041, Accuracy: 52277/60000 (87.13%)
Test set: Average loss: 0.1128, Accuracy: 9733/10000 (97.33%)

epoch number  2
Train set: Average loss: 0.0007, Accuracy: 58636/60000 (97.73%)
Test set: Average loss: 0.0749, Accuracy: 9808/10000 (98.08%)

epoch number  3
Train set: Average loss: 0.0005, Accuracy: 58982/60000 (98.30%)
Test set: Average loss: 0.0482, Accuracy: 9862/10000 (98.62%)

epoch number  4
Train set: Average loss: 0.0004, Accuracy: 59137/60000 (98.56%)
Test set: Average loss: 0.0430, Accuracy: 9880/10000 (98.80%)

epoch number  5
Train set: Average loss: 0.0004, Accuracy: 59238/60000 (98.73%)
Test set: Average loss: 0.0402, Accuracy: 9883/10000 (98.83%)

epoch number  6
Train set: Average loss: 0.0003, Accuracy: 59305/60000 (98.84%)
Test set: Average loss: 0.0370, Accuracy: 9891/10000 (98.91%)

epoch number  7
Train set: Average loss: 0.0003, Accuracy: 59345/60000 (98.91%)
Test set: Average loss: 0.0360, Accuracy: 9904/10000 (99.04%)


In [33]:
# fig = plt.figure(figsize=(20, 8))

# import numpy as np
# import matplotlib.pyplot as plt

# for idx in np.arange(10):
#   ax = fig.add_subplot(2, 10/2, idx+1, xticks=[], yticks=[])
#   img = incorrect_examples[idx][idx]
#   img = img/2 + 0.5
#   img = np.clip(img, 0, 1)
#   plt.imshow(img)